In [27]:
import re
import pandas as pd 
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
import time
import json
from PIL import Image
import pytesseract
import pathlib

In [28]:
tesseractPath = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
coveragecodes = ""
user = ""

In [29]:
# get all airport ID's
def get_airportList(file):
    df = pd.read_csv(file,usecols=["DestinationAirport"])
    df = df.sort_values(by=['DestinationAirport']).rename(columns={"DestinationAirport": "airport"})
    df = df.drop_duplicates(subset=['airport'])
    airportList = df['airport'].tolist()
    return airportList

# get airport ID for each airport
def get_airportID(apt):
    URL = "https://ww1.jeppesen.com/icharts/airports-json.jsp?coveragecodes="+ coveragecodes + "&search=" + apt + "&user="+ user
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    site_json = json.loads(soup.text)
    if site_json['Airports']['totalItems'] > 0:
        site_json = site_json['Airports']['items'][0]['airportId']
        print("Added AirportID: " + apt + '...')
    else:
        print("No_Airport_info: " + apt + '...' )
        site_json = 0
    return site_json

# get all doc ID for given airport
def get_documentID(apt,aptID):
    df = pd.DataFrame()
    URL = "https://ww1.jeppesen.com/icharts/charts-json.jsp?user="+ user + "&coveragecodes=" + coveragecodes + "&selectedcoveragecodes="+coveragecodes+"&icao="+ str(apt) +"&airport-id=" + str(aptID)
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    site_json = json.loads(soup.text)
    site_json = site_json['Charts']['items']
    output_dict = [x for x in site_json if x['category'] == 'APPROACH']
    if len(output_dict) > 0:
        for dID in range(len(output_dict)):
            df = df.append({'airport': apt
                            ,'airportID': aptID
                            ,'documentID': output_dict[dID]['docId']
                            ,'approachName': output_dict[dID]['procId']
                            ,'revDate': output_dict[dID]['revDate']
                            ,'effDate': output_dict[dID]['effDate']
                            },ignore_index=True)
        print("Added DocumanetIDs: " + str(apt + '...'))
    else:
        print("No Documents: " + str(apt + '...'))
    return df

# parase runway information
def parse_docRuwayinfo(df_approachs):
    df_approachs['Runway'] = df_approachs['approachName'].str.extract(r"(RWY.*)",expand=False)
    df_approachs = df_approachs[df_approachs['Runway'].notnull()]
    if 'CAT' in df_approachs['Runway']:
        df_approachs[['Runway', 'CAT']] = df_approachs['Runway'].str.split('CAT',expand=True)
        df_approachs = df_approachs.drop(columns=['CAT'])
    df_approachs[['name', 'RWY']] = df_approachs['Runway'].str.split('RWY',expand=True)
    df_approachs['RWY'] = df_approachs['RWY'].str.replace(' ','')
    df_approachs['RWY'] = df_approachs['RWY'].apply(lambda x: '0'+ x if len(x) < 2 else x )
    df_approachs['aptRWY'] = df_approachs['airport'] + '-RW'+df_approachs['RWY']
    df_approachs['documentID'] = df_approachs['documentID'].astype(int)
    df_approachs = df_approachs.drop(columns=['name','Runway'])
    return df_approachs

# use doc ID to do get to chart png file and save as png file
def get_savedCharts(df_approachesFlownto):
    np_chart = pd.DataFrame()
    for i,row in df_approachesFlownto.iterrows():
        time.sleep(5)
        URL = "https://ww1.jeppesen.com/icharts/tile?docid=" + str(df_approachesFlownto['documentID'][i]) +"&x=0&y=0&vpwidth=539&vpheight=945&twidth=539&theight=945&wholechart=true&rotation=0&zoom=7"
        img = Image.open(requests.get(URL, stream = True).raw)
        pytesseract.pytesseract.tesseract_cmd = tesseractPath
        text = pytesseract.image_to_string(img)
        text = text.rsplit('\n')[0]
        if text == 'Chart Not Available':
            np_chart = np_chart.append(row,ignore_index=True)
            print("No chart Found: " + str(df_approachesFlownto['documentID'][i]) + '_' + str(df_approachesFlownto['aptRWY'][i]) + '_' + str(df_approachesFlownto['approachName'][i]) )
        else:
            file = pathlib.Path('ApproachImages/' + str(df_approachesFlownto['aptRWY'][i]) + '_' + str(df_approachesFlownto['approachName'][i]) + '.png')
            if file.exists():
                print("Chart Exists: " + str(df_approachesFlownto['aptRWY'][i]) + '_' + str(df_approachesFlownto['approachName'][i]) + '...')
            else:
                img.save('ApproachImages/' + str(df_approachesFlownto['aptRWY'][i]) + '_' + str(df_approachesFlownto['approachName'][i]) + '.png')
                print("Added Chart: " + str(df_approachesFlownto['aptRWY'][i]) + '_' + str(df_approachesFlownto['approachName'][i]) + '...')
            
    return np_chart

In [31]:
# get airport list
apt_list = get_airportList('airport-rwy.csv')
apt_completed_df = pd.DataFrame()

file = pathlib.Path("apt_completed.csv")
if file.exists():
    apt_completed = get_airportList('apt_completed.csv')
else:
    apt_completed = []

# compare if apt was already done
if apt_completed:
    apt_list = [i for i in apt_list if i not in apt_completed]
    
# get unique runways we fly into
df_alnApt = pd.read_csv('airport-rwy.csv',usecols=["DestinationAirportRunway"])
df_alnApt = df_alnApt.sort_values(by=['DestinationAirportRunway']).rename(columns={"DestinationAirportRunway": "aptRWY"})

for apt in apt_list:
    # get airport ID for each airport
    time.sleep(5)
    apt_ID = get_airportID(apt)
    
    if apt_ID == 0:
        print("No Airport: " + apt)
        continue
    else:
        # get all doc ID for given airport
        time.sleep(5)
        df_doc = get_documentID(apt,apt_ID)
        
        if df_doc.empty:
            print("No Document: " + apt)
            continue
        else:
            # filter approaches
            df_doc = df_doc[df_doc['approachName'].str.contains("ILS|RNAV|LOC|RNP")]
            
            # parase runway information
            df_docRWY = parse_docRuwayinfo(df_doc)

            # merge inner to get only runways flown to
            df_usedRwys = df_alnApt.merge(df_docRWY, how='inner', on='aptRWY')

            # use doc ID to get to chart png file and save as png file
            no_chartsList = get_savedCharts(df_usedRwys)
            
            # add apt to completed list
            apt_completed_df = apt_completed_df.append({'DestinationAirport': apt},ignore_index=True)
            apt_completed_df.to_csv('apt_completed.csv')
            print("Completed: " + apt)

No_Airport_info: KAFW...
No Airport: KAFW
No_Airport_info: KMZJ...
No Airport: KMZJ
Added AirportID: MMSD...
Added DocumanetIDs: MMSD...
Added Chart: MMSD-RW16_RNP RWY 16...
Added Chart: MMSD-RW34_RNP RWY 34...
Completed: MMSD
Added AirportID: MMUN...
Added DocumanetIDs: MMUN...
Added Chart: MMUN-RW12L_ILS DME OR LOC 1 RWY 12L...
Added Chart: MMUN-RW12L_ILS DME OR LOC 2 RWY 12L...
Added Chart: MMUN-RW12L_ILS DME OR LOC 4 RWY 12L...
Added Chart: MMUN-RW12L_RNP 2 RWY 12L...
Added Chart: MMUN-RW12R_ILS DME OR LOC 1 RWY 12R...
Added Chart: MMUN-RW12R_ILS DME OR LOC 2 RWY 12R...
Added Chart: MMUN-RW12R_ILS DME OR LOC 4 RWY 12R...
Added Chart: MMUN-RW12R_RNP 2 RWY 12R...
Added Chart: MMUN-RW30L_RNP 2 RWY 30L...
Added Chart: MMUN-RW30R_RNP 2 RWY 30R...
Completed: MMUN
Added AirportID: MNMG...
Added DocumanetIDs: MNMG...
Added Chart: MNMG-RW10_ILS 1 RWY 10...
Added Chart: MNMG-RW10_ILS 2 RWY 10...
Added Chart: MNMG-RW10_ILS DME OR LOC RWY 10...
Added Chart: MNMG-RW10_RNAV (GNSS) RWY 10...
Comp